In [82]:
import numpy as np 
import pandas as pd 

In [4]:
movies_data = pd.read_csv("../input/movielens100k/movies.csv")
movies_data.drop_duplicates(subset ="title",keep='first',inplace=True,ignore_index=True) 
movies_data.head() 

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
ratings_data = pd.read_csv("../input/movielens100k/ratings.csv")
ratings_data.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [6]:
movies_with_ratings = movies_data.merge(ratings_data, on='movieId').reset_index(drop=True)
movies_with_ratings.dropna(inplace=True)
movies_with_ratings.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,7,3.0,851866703
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,9,4.0,938629179
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,13,5.0,1331380058
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,15,2.0,997938310
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,19,3.0,855190091


In [7]:
from surprise import KNNWithMeans, KNNBasic, KNNWithZScore, KNNBaseline
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise.model_selection import GridSearchCV

In [8]:
dataset = pd.DataFrame({
    'uid': movies_with_ratings.userId,
    'iid': movies_with_ratings.title,
    'rating': movies_with_ratings.rating
})

In [9]:
dataset.head()

,uid,iid,rating
0,7,Toy Story (1995),3.0
1,9,Toy Story (1995),4.0
2,13,Toy Story (1995),5.0
3,15,Toy Story (1995),2.0
4,19,Toy Story (1995),3.0


In [10]:
dataset.rating.min(), dataset.rating.max() 

(0.5, 5.0)

In [11]:
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(dataset, reader)

In [12]:
algos = (
    KNNWithZScore(),
    KNNBasic(),
    KNNWithMeans(),
    KNNBaseline(),
 )

In [25]:
# перебор алгоритмов семейства KNN со стандартными параметрами
benchmark_kNN = []
for algo in algos:
    results = cross_validate(algo, data, measures=['RMSE'], cv=5,verbose = False)
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algo).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    benchmark_kNN.append(tmp)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.


/tmp/ipykernel_32/4247557125.py:5: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp = tmp.append(pd.Series([str(algo).split(' ')[0].split('.')[-1]], index=['Algorithm']))


Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.


/tmp/ipykernel_32/4247557125.py:5: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp = tmp.append(pd.Series([str(algo).split(' ')[0].split('.')[-1]], index=['Algorithm']))


Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.


/tmp/ipykernel_32/4247557125.py:5: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp = tmp.append(pd.Series([str(algo).split(' ')[0].split('.')[-1]], index=['Algorithm']))


Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.


/tmp/ipykernel_32/4247557125.py:5: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp = tmp.append(pd.Series([str(algo).split(' ')[0].split('.')[-1]], index=['Algorithm']))


In [26]:
# результаты алгоритмов KNN
knn_results = pd.DataFrame(benchmark_kNN).set_index('Algorithm').sort_values('test_rmse')
knn_results

,test_rmse,fit_time,test_time
Algorithm,,,
KNNBaseline,0.898308,0.489194,2.971005
KNNWithMeans,0.919687,0.200726,2.596405
KNNWithZScore,0.920743,0.305337,2.792748
KNNBasic,0.968528,0.155610,2.336151


In [29]:
from surprise import SVD, SlopeOne, NMF, CoClustering
algos = (
    SVD(),
    SlopeOne(),
    NMF(),
    CoClustering()
 )

In [30]:
# перебор остальных других алгоритмов коллаборативной фильтрации
benchmark_others = []
for algo in algos:
    results = cross_validate(algo, data, measures=['RMSE'], cv=5,verbose = False)
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algo).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    benchmark_others.append(tmp)

/tmp/ipykernel_32/4202431634.py:5: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp = tmp.append(pd.Series([str(algo).split(' ')[0].split('.')[-1]], index=['Algorithm']))
/tmp/ipykernel_32/4202431634.py:5: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp = tmp.append(pd.Series([str(algo).split(' ')[0].split('.')[-1]], index=['Algorithm']))
/tmp/ipykernel_32/4202431634.py:5: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp = tmp.append(pd.Series([str(algo).split(' ')[0].split('.')[-1]], index=['Algorithm']))
/tmp/ipykernel_32/4202431634.py:5: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tmp = tmp.append(pd.Series([str(algo).split

In [32]:
# результаты других алгоритмов
others_results = pd.DataFrame(benchmark_others).set_index('Algorithm').sort_values('test_rmse')
others_results

,test_rmse,fit_time,test_time
Algorithm,,,
SVD,0.896281,1.471502,0.291192
SlopeOne,0.930407,4.647362,7.899845
NMF,0.947569,3.848470,0.226168
CoClustering,0.968121,3.568070,0.215958


In [40]:
# подбор гиперпараметров для SVD
param_grid = {
    "n_epochs": [50, 80, 100],
    "lr_all": [0.005, 0.02],
    "reg_all": [0.05],
    "n_factors":[100, 150, 200],
    'random_state' : [8]
}
gs_SVD = GridSearchCV(SVD, param_grid, measures=["rmse"], refit=True, cv=5)

gs_SVD.fit(data)

In [41]:
print("BEST RMSE: \t", gs_SVD.best_score["rmse"])
print("BEST params: \t", gs_SVD.best_params["rmse"])

BEST RMSE: 	 0.8774995312245613
BEST params: 	 {'n_epochs': 50, 'lr_all': 0.02, 'reg_all': 0.05, 'n_factors': 200}


### Попробуем получить результат лучше на модели семейства KNN. Возьмем лучшую из них и проведем тюнинг параметров модели

In [33]:
# перебор метрик близости для KNN, а также выбор между user_based и item_based
sim_options = {
    "name": ['cosine','pearson','pearson_baseline'],
    "user_based": [True,False]
}

In [38]:
param_grid_knn = {'sim_options' : sim_options}
gs_knn = GridSearchCV(KNNBaseline, param_grid_knn, measures=["rmse"],random_state = 8, cv=5)
gs_knn.fit(data)

Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the cosine similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Comput

In [39]:
print("BEST RMSE: \t", gs_knn.best_score["rmse"])
print("BEST params: \t", gs_knn.best_params["rmse"])

BEST RMSE: 	 0.8738761135809401
BEST params: 	 {'sim_options': {'name': 'pearson_baseline', 'user_based': False}}


In [54]:
sim_options = {"name": ["pearson_baseline"], 
               "user_based": [False]
              } 

In [60]:
# подбор количества соседей
param_grid = {
    'sim_options' : sim_options,
    "k": [40,50,60,70],
    'random_state' : [8]
}
gs_knn = GridSearchCV(KNNBaseline, param_grid, measures=["rmse"], refit=True, cv=5)
gs_knn.fit(data)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline si

In [61]:
print("BEST RMSE: \t", gs_knn.best_score["rmse"])
print("BEST params: \t", gs_knn.best_params["rmse"])

BEST RMSE: 	 0.8747836965821086
BEST params: 	 {'sim_options': {'name': 'pearson_baseline', 'user_based': False}, 'k': 50}


In [73]:
# подбор параметров оптимизатора
sim_options = {"name": ["pearson_baseline"], 
               "user_based": [False]
              }
bsl_options = {
    "method": ["als"],
    'reg_i' : [2,5],
    'reg_u' : [5,10],
    'n_epochs' : [20,30]
    }

In [74]:
param_grid = {
    'sim_options' : sim_options,
    'bsl_options' : bsl_options,
    "k": [50],
    'random_state' : [8]
}
gs_knn = GridSearchCV(KNNBaseline, param_grid, measures=["rmse"],cv=5)
gs_knn.fit(data)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline si

In [75]:
print("BEST RMSE: \t", gs_knn.best_score["rmse"])
print("BEST params: \t", gs_knn.best_params["rmse"])

BEST RMSE: 	 0.8719359586013136
BEST params: 	 {'sim_options': {'name': 'pearson_baseline', 'user_based': False}, 'bsl_options': {'method': 'als', 'reg_i': 5, 'reg_u': 5, 'n_epochs': 30}, 'k': 50, 'random_state': 8}
